## An SVM fitting and testing code

<p>&nbsp;</p>

In this example we will:
- Generate random multivariate data from a mixture of two normal distributions
- Split the data into training and testing subsets
- Fit a linear SVM to the data
- Test the resulting classification model on the testing subset


In [ ]:
# Initial block to define basic parameters, generate the data and plot it

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#np.random.seed(1)

## Define numbers of observations and variables

V = 25    # number of variables (add one to account for the independent term)
N = 500  # number of observations

## Other parameters

mu = 4       # distance between centers of observations
s_min = 0.5  # min scaling parameter
s_max = 2    # max scaling parameter

C_SVM = 2    # SVM parameter

### Assign sizes of the two groups

N0 = np.random.randint(0.1*N,0.9*N)
N1 = N - N0

## Generate a matrix of normal random values

X = np.random.normal(0,1,size=([N,V]))

Y = np.ones((N,1))
Y[ 0:N0 ] = -1

### For one of the groups, separate centers by mu*u_0 (approximately) and scale by v_0

u_0 = np.random.normal(0,1,size=(V,1))
u_0 = u_0/np.linalg.norm(u_0,ord=2)

v_0 = np.random.uniform(s_min,s_max,size=(1,V))

X[ N0:N , :] = X[ N0:N , :] * v_0[ : , None ]
X[ N0:N , :] = X[ N0:N , :] + mu*u_0.T[ : , None]

## Plot observations along u_0 and an orthogonal direction u_1

e_vec = np.ones((1,V))
u_1 = e_vec.T - np.dot(e_vec,u_0)*u_0
u_1 = u_1/np.linalg.norm(u_1,ord=2)

plt.figure(figsize=(6, 6))
plt.plot(np.dot(X[ 0:N0 , : ],u_0),np.dot(X[ 0:N0 , : ],u_1),'ro')
plt.plot(np.dot(X[ N0:N , : ],u_0),np.dot(X[ N0:N , : ],u_1),'bo')

## Show plot

plt.show()


In [ ]:
# Create two subsets of observations, to be used as training and testing sets

N_tr = np.rint(N*0.8).astype(int)
N_ts = N - N_tr

## Random permutation to assign observations and indices for both subsets

ix_0 = np.random.permutation(np.arange(N))

ix_tr = ix_0[0:N_tr]
ix_ts = ix_0[N_tr:N]

## Matrices with data for each subset

X_tr = X[ix_tr,:]
X_ts = X[ix_ts,:]

Y_tr = Y[ix_tr]
Y_ts = Y[ix_ts]


Define and solve the dual SVM problem,

$$
  \begin{array}{lc}
    \max_{\alpha} & \sum_i \alpha_i - \frac{1}{2} \sum_i \sum_j \alpha_i \alpha_j y_i y_j x_i^T x_j \\
    \mbox{s.t.} & \sum_i y_i \alpha_i = 0 \\
    & 0 \leq \alpha_i \leq C
  \end{array}
$$

Note that we have to minimize the objective function in order to use the *minimize* library


In [ ]:
# Define objective function and its first derivatives for the SVM model

def svm_obj(alpha,X,Y):
    alpha = np.matrix(alpha).T
    alpha_y = np.multiply(alpha,Y)
    quad_tr = np.dot(X.T,alpha_y)
    quad_t = np.dot(quad_tr.T,quad_tr)
    lin_t = np.sum(alpha)
    ff = 0.5*quad_t - lin_t
    ff = ff.item(0)
    return ff

def svm_grad(alpha,X,Y):
    alpha = np.matrix(alpha).T
    alpha_y = np.multiply(alpha,Y)
    quad_tr0 = np.dot(X.T,alpha_y)
    quad_tr1 = np.dot(X,quad_tr0)
    quad_t = np.multiply(Y,quad_tr1)
    gg = quad_t - 1
    gg = np.squeeze(np.asarray(gg))
    return gg


In [ ]:
# Check that the derivative routine for the objective function is computing correct values

## Initial values for  alpha, u and delta

alpha_0 = np.ones(N_tr)
delta_v = 1.0e-4
u_p = np.random.normal(0,1,size=N_tr)
u_p = u_p/np.linalg.norm(u_p,ord=2)

## Values of the objective function at  alpha  and  alpha + delta*u

alpha_1 = alpha_0 + delta_v*u_p

v_1 = svm_obj(alpha_1,X_tr,Y_tr)
v_0 = svm_obj(alpha_0,X_tr,Y_tr)

## Compute the gradient at  alpha

grad_0 = svm_grad(alpha_0,X_tr,Y_tr)

## Compare the value of the directional derivative  g'u  with the 
## finite difference approximation  (f_1 - f_0)/delta

fd_appr = (v_1 - v_0)/delta_v
dir_der = np.dot(grad_0.T,u_p)
print('Finite difference approx: %12.6f' %fd_appr)
print('Directional derivative:   %12.6f' %dir_der)


In [ ]:
# Define constraint and bounds for the SVM model

from scipy.optimize import minimize
from scipy.optimize import Bounds

## Define the constraint

eq_cns = { 'type': 'eq',
           'fun' : lambda alpha,X,Y : alpha.dot(Y) ,
           'args': (X_tr,Y_tr) }

## Two auxiliary vectors, a vector of ones and a vector of zeros

vct_e = np.ones(N_tr)
vct_0 = np.zeros(N_tr)

# Define bounds

bnds = Bounds(vct_0, C_SVM*vct_e)


In [ ]:
# Use constrained optimization procedure (SLSQP,trust-constr,etc) to optimize the SVM model
# Use training data subset

import time

## Initial values for the dual variables, alpha

alpha0 = np.ones(N_tr)

## Call the optimization routine

time_start = time.process_time()

res = minimize(svm_obj,alpha0,args=(X_tr,Y_tr),jac=svm_grad,bounds=bnds,constraints=eq_cns,options={'disp': True})

time_elapsed = (time.process_time() - time_start) 

## Print results

print('Elapsed time = %8.5f' %(time_elapsed))


From the solution of the optimization problem, we define a separating hyperplane such that

$$
  \begin{array}{cl}
    \mbox{if}\quad w^T x + b \geq 0 & \mbox{ then } x \in A \\
    \mbox{if}\quad w^T x + b < 0 & \mbox{ then } x \in B
  \end{array}
$$

The value of $w$ can be obtained from the primal-dual relationships as

$$
   w = \sum_i \alpha_i y_i x_i
$$

The value of $b$ can be obtained from a support vector $x_j$ (a vector with $j$ satisfying $C > \alpha_j > 0$) as

$$
   b = y_j - w^T x_j
$$

or as the average of the values $b$ for all support vectors.


In [ ]:
# Extract values defining the separating hyperplane, w and b

alpha_sv = res.x

## Value of w from its formula

w_val = np.dot(np.multiply(alpha_sv,Y_tr.T),X_tr)

## Value of b (b_val) as the mean of the computed values for all support vectors

l_sv = (alpha_sv > 1.0e-4) & (alpha_sv < C_SVM - 1.0e-4)
i_sv = np.array(range(len(alpha_sv)))
i_sv = i_sv[l_sv]

b_val = np.mean(Y_tr[i_sv] - np.dot(X_tr[i_sv,:],w_val.T))


In [ ]:
# Represent the separating hyperplane on a 2-dimensional subspace

## Plot the observations

plt.figure(figsize=(6, 6))

x_u0 = np.dot(X,u_0)
x_u1 = np.dot(X,u_1)
x_a_u0 = x_u0[0:N0]
x_a_u1 = x_u1[0:N0]
x_b_u0 = x_u0[N0:N]
x_b_u1 = x_u1[N0:N]

plt.plot(x_a_u0,x_a_u1,'ro')
plt.plot(x_b_u0,x_b_u1,'bo')

## Plot the separating hyperplane

x_mn = np.min(x_u0)
x_mx = np.max(x_u0)
y_mn = np.min(x_u1)
y_mx = np.max(x_u1)

x_plt = np.arange(x_mn,x_mx,(x_mx-x_mn)/100)

a_0 = np.dot(w_val,u_0).item(0)
a_1 = np.dot(w_val,u_1).item(0)
y_plt = -(b_val + a_0*x_plt)/a_1

l_plt = (y_plt < y_mx) & (y_plt > y_mn)

plt.plot(x_plt[l_plt],y_plt[l_plt],'g')

## Show plot

plt.show()


In [ ]:
# Check the quality of the results on the testing subset

## Compute the scores from the SVM

cls_sv_ts = np.dot(X_ts,w_val.T) + b_val

## Compare the positive scores with the values of Y_ts

l_pos = (cls_sv_ts >= 0)
Y_pos = Y_ts[l_pos]
n_a = len(Y_pos)
l_pos_a = (Y_pos == 1)

cnt_aa = len(Y_pos[l_pos_a]) 
cnt_ab = n_a - cnt_aa

## Compare the negative scores with the values of Y_ts

l_neg = (cls_sv_ts < 0)
Y_neg = Y_ts[l_neg]
n_b = len(Y_neg)
l_neg_a = (Y_neg == -1)

cnt_bb = len(Y_neg[l_neg_a])
cnt_ba = n_b - cnt_bb

## Print the results for the numbers of correct and incorrect idenfications

print('                Generated (correct) labels')
print('                          A       B ')
print(' Identified label A: %6d  %6d' %(cnt_aa,cnt_ab))
print(' Identified label B: %6d  %6d' %(cnt_ba,cnt_bb))
